# Usando o algoritmo customizado no Amazon SageMaker

## Criando o setup do ambiente

In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role


prefix = 'martinig-sagemaker'
container_image_uri = '206730628141.dkr.ecr.us-east-1.amazonaws.com/dummytest:latest'
role = get_execution_role()
account_id = role.split(':')[4]
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
bucket = sagemaker_session.default_bucket()

print(account_id)
print(region)
print(role)
print(bucket)

## Gerando um dataset de exemplo

In [ ]:
!echo "val1, val2, val3" > data/dummy.csv
print(sagemaker_session.upload_data('data/dummy.csv', bucket, prefix + '/train'))
print(sagemaker_session.upload_data('data/dummy.csv', bucket, prefix + '/val'))
!rm data/dummy.csv

## Criando o Estimator do Amazon SageMaker e iniciando o treino

In [ ]:
est = sagemaker.estimator.Estimator(container_image_uri,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='local', # use local mode
                                    #train_instance_type='ml.m5.xlarge',
                                    base_job_name=prefix)

est.set_hyperparameters(hp1='value1',
                        hp2=300,
                        hp3=0.001)

train_config = sagemaker.session.s3_input('s3://{0}/{1}/train/'.format(bucket, prefix), content_type='text/csv')
val_config = sagemaker.session.s3_input('s3://{0}/{1}/val/'.format(bucket, prefix), content_type='text/csv')

est.fit({'train': train_config, 'validation': val_config })